In [226]:
#import desired libraries
import pandas as pd
import numpy as np
import string
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
import spacy
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [227]:
tokenizer=ToktokTokenizer()
#en_core_web_sm is a small-sized pre-trained model optimized for various NLP tasks
#disable=['ner']: This parameter is used to disable certain components of the loaded model.
nlp=spacy.load('en_core_web_sm',disable=['ner'])

**Polarity and Subjectivity**

In [228]:
#scale sentiment of each sentence
#list sentences and use every sentence in textblob.sentiment
#polarity from [-1:1] positive and negative sentiment
#subjectivity stands for public (factual information) opion or not from [0:1]
text=["he is very good boy","he is a not good boy","everybody says this man is poor"]
i=0
for sen in text :
  i+=1
  print(f"for sentence {i} : ",TextBlob(sen).sentiment)
  print()

for sentence 1 :  Sentiment(polarity=0.9099999999999999, subjectivity=0.7800000000000001)

for sentence 2 :  Sentiment(polarity=-0.35, subjectivity=0.6000000000000001)

for sentence 3 :  Sentiment(polarity=-0.4, subjectivity=0.6)



In [229]:
#import data
#read data from directory
#distinguish data
#review and sentiment
#you can copy the path by copy path option
data=pd.read_csv('../data/interim/Train.csv',error_bad_lines=False)
df=data.copy()
#first 5 rows
df.head()

<ipython-input-229-324e90a86e76>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data=pd.read_csv('../content/sample_data/Train.csv',error_bad_lines=False)


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [230]:
df.shape

(40000, 2)

In [231]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
text,40000,39723,"Hilarious, clean, light-hearted, and quote-wor...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
label,40000.0,NaN,NaN,NaN,0.499525,0.500006,0.0,0.0,0.0,1.0,1.0


In [232]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [234]:
#lowering data size to 5000 from each labale
df_0=df[df['label'] == 0].sample(n=5000)
df_1=df[df['label'] == 1].sample(n=5000)

In [235]:
#by default axis=0 so you donot need to write it again
df=pd.concat([df_0,df_1])
#apply shuffling
df =shuffle(df)
df

,text,label
20552,"Nifty little episode played mainly for laughs,...",1
38647,It amazes me that someone would actually consi...,0
21294,"I went to see this 3 nights ago here in Cork, ...",0
13573,I first saw this as a kid (THE LITTLE RASCALS ...,1
18907,Me and a couple of friends went to rent some m...,0
...,...,...
8138,"I have a severe problem with this show, severa...",0
18303,I first remember bumping into this zaniness fr...,1
18040,The first 10 minutes of the movie makes fun of...,0
14306,Set in Bam Margera's hometown of Westchester P...,1


**Data Preprocessing**

In [236]:
#string operation
#remove special pattern of our data using regex
df.replace(r'^\s*$', np.nan, regex=True,inplace=True)
#drop na values after replacement
df.dropna(axis = 0, how = 'any', inplace = True)

In [237]:
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [238]:
#check for na values
df.isna().sum()

text     0
label    0
dtype: int64

In [239]:
#normalize text to ascii
#ignore non ascii text ---> encode text to ascii ignore other and then decode
#the only column we have for text and the other for labelling
df['text']=df['text'].str.encode('ascii','ignore').str.decode('ascii')

In [240]:
#remove punctuation
def remove_punc(text):
  #loop on all punctation and if we find symbol in sentence we replace it with nothin
  for punc in string.punctuation :
    text=text.replace(punc,"")
  return text
df['text']=df['text'].apply(remove_punc)

In [241]:
df

,text,label
20552,Nifty little episode played mainly for laughs ...,1
38647,It amazes me that someone would actually consi...,0
21294,I went to see this 3 nights ago here in Cork I...,0
13573,I first saw this as a kid THE LITTLE RASCALS f...,1
18907,Me and a couple of friends went to rent some m...,0
...,...,...
8138,I have a severe problem with this show several...,0
18303,I first remember bumping into this zaniness fr...,1
18040,The first 10 minutes of the movie makes fun of...,0
14306,Set in Bam Margeras hometown of Westchester PA...,1


In [242]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [243]:
#remove stop words
#tokenization is important for removing stop words later as comparing them with list of stop words later
#create tokenizer object ---> this is replcement for token_word
token=ToktokTokenizer()
#donot forget nltk.download
from nltk.corpus import stopwords
#stop words in english
stopwords=nltk.corpus.stopwords.words('english')
#remove no and not from stop words because we need them in sentiment analysis and this affect our prediction ---> it is list
stopwords.remove('no')
stopwords.remove('not')

def remove_stopwords(text , is_lower_case = False) :
  #text maybe small or capital
  tokens=token.tokenize(text)
  #remove any white blank for every elemnt for list of words using list comprehensive
  tokens=[i.strip() for i in tokens]
  if is_lower_case :
    new_tokens = [i for i in  tokens if i not in stopwords]
  else :
    new_tokens = [i for i in tokens if i.lower() not in stopwords]
  #return filtered tokens converting list to string
  filtered_tokens=' '.join(new_tokens)
  return filtered_tokens
df['text']=df['text'].apply(remove_stopwords)

In [244]:
df

,text,label
20552,Nifty little episode played mainly laughs clev...,1
38647,amazes someone would actually consider spendin...,0
21294,went see 3 nights ago Cork Ireland world premi...,0
13573,first saw kid LITTLE RASCALS first went TV yea...,1
18907,couple friends went rent movies one day picked...,0
...,...,...
8138,severe problem show several actually simple li...,0
18303,first remember bumping zaniness Zucker brother...,1
18040,first 10 minutes movie makes fun sequels pg13 ...,0
14306,Set Bam Margeras hometown Westchester PA Hagga...,1


In [245]:
#again remove special characters
def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text
df['text']=df['text'].apply(remove_special_characters)

In [246]:
#remove html
#parse html code and erase special pattern using beautiful soup and regex
#beautiful soup is not only for web scrapping but aslo for html parsing basicly
def remove_html(text) :
  soup=BeautifulSoup(text,"html.parser")
  text=soup.get_text()
  #remove special charcters
  text=re.sub('\[[^]]*\]','',text)
  return text
df['text']=df['text'].apply(remove_html)

In [247]:
#remove url
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r' ',text)
df['text']=df['text'].apply(remove_URL)

In [248]:
def remove_numbers(text):
    #remove integers
    text = ''.join([i for i in text if not i.isdigit()])
    return text
df['text']=df['text'].apply(remove_numbers)

In [249]:
#remove alphanumeric so to do it first we identify these using regex then clean it
#function word to detect word
def cleanse(word):
    #compile regualar expression to regular object
    #the regular expression pattern \D*\d matches any sequence of zero or more non-digit characters followed by exactly one digit character.
    rx = re.compile(r'\D*\d')
    #check if it matches the words or not
    #if matches return nothing to the removal function if it is not then return word
    if rx.match(word):
        return ''
    return word
def remove_alphanumeric(strings):
    nstrings = [" ".join(filter(None, (
    cleanse(word) for word in string.split())))
    for string in strings.split()]
    str1 = ' '.join(nstrings)
    return str1
df['text']=df['text'].apply(remove_alphanumeric)

In [250]:
#lemmatization
def lemmatize_text(text):
    #store text with NLP
    text = nlp(text)
    #if it is bad word return it as it without lemmatization
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
df['text']=df['text'].apply(lemmatize_text)

In [251]:
#add sentiment column depeneds on text it self
def add_sentiment(text):
  return TextBlob(text).sentiment
#here we get tuple for every instance consists of polarity and subjectivity
df['sentiment']=df['text'].apply(add_sentiment)

In [252]:
#get overview info about df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 20552 to 13570
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       10000 non-null  object
 1   label      10000 non-null  int64 
 2   sentiment  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 312.5+ KB


In [253]:
df

,text,label,sentiment
20552,nifty little episode play mainly laugh clever ...,1,"(0.11661255411255413, 0.5225108225108225)"
38647,amaze someone would actually consider spend mo...,0,"(0.0271978021978022, 0.5485042735042733)"
21294,go see night ago Cork Ireland world premiere t...,0,"(0.21223639455782317, 0.5810034013605442)"
13573,first see kid LITTLE rascal first go tv year b...,1,"(0.14027777777777778, 0.4629629629629629)"
18907,couple friend go rent movie one day pick one o...,0,"(0.0, 0.0)"
...,...,...,...
8138,severe problem show several actually simple li...,0,"(0.07390395552160257, 0.4951404804345981)"
18303,first remember bump zaniness Zucker brother Ji...,1,"(0.053406084656084665, 0.5951388888888888)"
18040,first minute movie make fun sequel pg movie re...,0,"(-0.00249999999999998, 0.6733333333333335)"
14306,Set Bam Margeras hometown Westchester PA Hagga...,1,"(-0.11416666666666668, 0.5741666666666667)"


In [254]:
#sperate polarity and subjectivity as columns for df
#we can do this by add new df with two columns then concat it and then drop sentiment
sentiment_lst=df['sentiment'].to_list()
sentiment_lst

[Sentiment(polarity=0.11661255411255413, subjectivity=0.5225108225108225),
 Sentiment(polarity=0.0271978021978022, subjectivity=0.5485042735042733),
 Sentiment(polarity=0.21223639455782317, subjectivity=0.5810034013605442),
 Sentiment(polarity=0.14027777777777778, subjectivity=0.4629629629629629),
 Sentiment(polarity=0.0, subjectivity=0.0),
 Sentiment(polarity=0.06944444444444445, subjectivity=0.6583333333333333),
 Sentiment(polarity=0.07587412587412588, subjectivity=0.5220529470529469),
 Sentiment(polarity=-0.36666666666666664, subjectivity=0.5),
 Sentiment(polarity=-0.05531593406593406, subjectivity=0.4675343406593407),
 Sentiment(polarity=0.007936507936507933, subjectivity=0.39550264550264547),
 Sentiment(polarity=0.33333333333333337, subjectivity=0.7416666666666667),
 Sentiment(polarity=-0.23383928571428575, subjectivity=0.6422023809523808),
 Sentiment(polarity=0.18333333333333332, subjectivity=0.38749999999999996),
 Sentiment(polarity=-0.08237179487179486, subjectivity=0.526282051

In [255]:
columns=['polarity', 'subjectivity']
#to avoid duplication and rows with na values add index=df.index
#ensure retains the same row indices as the original DataFrame
df1=pd.DataFrame(sentiment_lst,columns=columns,index=df.index)
df1

,polarity,subjectivity
20552,0.116613,0.522511
38647,0.027198,0.548504
21294,0.212236,0.581003
13573,0.140278,0.462963
18907,0.000000,0.000000
...,...,...
8138,0.073904,0.495140
18303,0.053406,0.595139
18040,-0.002500,0.673333
14306,-0.114167,0.574167


In [256]:
#concat df with df1
df=pd.concat([df,df1],axis=1)
#fix sentiment column to positive or negative instead of numbers based on polarity
#filter df for our polarity threshold = 0.3
df.loc[df['polarity'] >= 0.3]['sentiment'] = 'Positive'
df.loc[df['polarity'] < 0.3]['sentiment'] = 'Nositive'

<ipython-input-256-1277047c4ec0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['polarity'] >= 0.3]['sentiment'] = 'Positive'
<ipython-input-256-1277047c4ec0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['polarity'] < 0.3]['sentiment'] = 'Nositive'


In [257]:
df

,text,label,sentiment,polarity,subjectivity
20552,nifty little episode play mainly laugh clever ...,1,"(0.11661255411255413, 0.5225108225108225)",0.116613,0.522511
38647,amaze someone would actually consider spend mo...,0,"(0.0271978021978022, 0.5485042735042733)",0.027198,0.548504
21294,go see night ago Cork Ireland world premiere t...,0,"(0.21223639455782317, 0.5810034013605442)",0.212236,0.581003
13573,first see kid LITTLE rascal first go tv year b...,1,"(0.14027777777777778, 0.4629629629629629)",0.140278,0.462963
18907,couple friend go rent movie one day pick one o...,0,"(0.0, 0.0)",0.000000,0.000000
...,...,...,...,...,...
8138,severe problem show several actually simple li...,0,"(0.07390395552160257, 0.4951404804345981)",0.073904,0.495140
18303,first remember bump zaniness Zucker brother Ji...,1,"(0.053406084656084665, 0.5951388888888888)",0.053406,0.595139
18040,first minute movie make fun sequel pg movie re...,0,"(-0.00249999999999998, 0.6733333333333335)",-0.002500,0.673333
14306,Set Bam Margeras hometown Westchester PA Hagga...,1,"(-0.11416666666666668, 0.5741666666666667)",-0.114167,0.574167


In [258]:
#rename label column to sentiment label using .rename function
df.rename(columns={'label': 'sentiment_label'}, inplace=True)
df

,text,sentiment_label,sentiment,polarity,subjectivity
20552,nifty little episode play mainly laugh clever ...,1,"(0.11661255411255413, 0.5225108225108225)",0.116613,0.522511
38647,amaze someone would actually consider spend mo...,0,"(0.0271978021978022, 0.5485042735042733)",0.027198,0.548504
21294,go see night ago Cork Ireland world premiere t...,0,"(0.21223639455782317, 0.5810034013605442)",0.212236,0.581003
13573,first see kid LITTLE rascal first go tv year b...,1,"(0.14027777777777778, 0.4629629629629629)",0.140278,0.462963
18907,couple friend go rent movie one day pick one o...,0,"(0.0, 0.0)",0.000000,0.000000
...,...,...,...,...,...
8138,severe problem show several actually simple li...,0,"(0.07390395552160257, 0.4951404804345981)",0.073904,0.495140
18303,first remember bump zaniness Zucker brother Ji...,1,"(0.053406084656084665, 0.5951388888888888)",0.053406,0.595139
18040,first minute movie make fun sequel pg movie re...,0,"(-0.00249999999999998, 0.6733333333333335)",-0.002500,0.673333
14306,Set Bam Margeras hometown Westchester PA Hagga...,1,"(-0.11416666666666668, 0.5741666666666667)",-0.114167,0.574167


In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 20552 to 13570
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             10000 non-null  object 
 1   sentiment_label  10000 non-null  int64  
 2   sentiment        10000 non-null  object 
 3   polarity         10000 non-null  float64
 4   subjectivity     10000 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 468.8+ KB
